# Imports

In [16]:
from dataclasses import dataclass, field
from typing import Optional

import pandas as pd
import torch

from federated.client.public_client import PublicClient
from federated.client.private_client import PrivateClient
from federated.server import Server
from data_loaders.mnist.data_loader import DataLoader
from models.mnist.mnist_cnn import MnistCNN as Model

mps_device = torch.device("mps")

# Set up parameter data class

In [17]:
@dataclass
class Params:
    n_clients: int = 10
    n_adv: int = 0
    n_epochs: int = 10
    protection: bool = False
    batch_size: int = 64
    is_private: bool = False
    epsilon: Optional[float] = None
    delta: Optional[float] = None

    def __post_init__(self):
        if self.is_private:
            if self.epsilon is None or self.delta is None:
                raise ValueError("When 'is_private' is True, 'epsilon' and 'delta' must be defined.")

params = Params()

# Set up Data Loader

In [18]:
dataloader = DataLoader(
        batch_size=params.batch_size,
        device=mps_device,
        test_split=0.2,
        val_split=0.2,
        n_clients=params.n_clients,
        num_adversaries=params.n_adv,
    )

# Set up Server and Clients

In [19]:
server = Server(
        model=Model(),
        device=mps_device,
        validation_data=dataloader.val_loader,
        enable_adversary_protection=params.protection,
    )

clients = [
        (
            PublicClient(
                id=f"{i}",
                model=Model(),
                device=mps_device,
                data=dataloader.train_loaders[i],
            )
            if not params.is_private
            else PrivateClient(
                id=f"{i}",
                model=Model(dropout=False),
                device=mps_device,
                data=dataloader.train_loaders[i],
                max_grad_norm=1.0,
                num_epochs=params.n_epochs,
                target_epsilon=params.epsilon,
                target_delta=params.delta,
            )
        )
        for i in range(params.n_clients)
    ]

# Run training

In [23]:
# Run 50 epochs on each client
for epoch in range(params.n_epochs):
    # train each client
    for i in range(params.n_clients):
        current_client = clients[i]
        current_client.update_parameters(server.model.state_dict())
        current_client.train_epoch()
        server.add_client_parameters(
            current_client.model.state_dict(),
            current_client.num_samples,
        )
    server.aggregate_parameters()
    if epoch % 2 == 0 or epoch == params.n_epochs - 1:
        server.evaluate_model()

Client 0 | Epoch 4: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 77.51it/s]
Client 0 | Train Accuracy: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 191.35it/s]


Client 0 | Epoch 5 | Loss: 0.3136 | Tr. Acc: 59.2281


Client 1 | Epoch 4: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 86.20it/s]
Client 1 | Train Accuracy: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 186.47it/s]


Client 1 | Epoch 5 | Loss: 0.3087 | Tr. Acc: 59.6140


Client 2 | Epoch 4: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 87.45it/s]
Client 2 | Train Accuracy: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 187.17it/s]


...

Client 8 | Epoch 14 | Loss: 0.1224 | Tr. Acc: 61.8596


Client 9 | Epoch 13: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 85.98it/s]
Client 9 | Train Accuracy: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 188.36it/s]


Client 9 | Epoch 14 | Loss: 0.1173 | Tr. Acc: 61.7719


Validating model: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 188/188 [00:01<00:00, 186.48it/s]

Accuracy of the network on the server validation images: 97.28%


# Run Evaluation

In [24]:
with torch.no_grad():
        server.model.eval()
        correct = 0
        total = 0
        for images, labels in dataloader.test_loader:
            images, labels = images.to(mps_device), labels.to(mps_device)
            outputs = server.model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(
            f"Accuracy of the network on the test images: {100 * correct / total:.2f}%"
        )

Accuracy of the network on the test images: 97.12%


# Save to CSV

In [ ]:
# save data to csv
all_dfs = []
for i in range(n_clients):
    all_dfs.append(clients[i].train_history)

# combine all dataframes to one dataframe
df = pd.concat(all_dfs)
client_save_name = f"project/results/n_clients_{n_clients}_n_adv_{n_adv}_enable_adv_{enable_adv_protection}_n_epochs_{num_epochs}_batch_size_{batch_size}_private_{should_use_private}_eps_{target_epsilon}_delta_{target_delta}.csv"
df.to_csv(client_save_name, index=False)
server_save_name = f"project/results/n_clients_{n_clients}_n_adv_{n_adv}_enable_adv_{enable_adv_protection}_n_epochs_{num_epochs}_batch_size_{batch_size}_private_{should_use_private}_eps_{target_epsilon}_delta_{target_delta}_server.csv"
server.val_performance.to_csv(server_save_name, index=False)